In [ ]:
# 编写ORM，实现通过对象来操作数据库
import asyncio,aiomysql
from email import charset
import logging

from itsdangerous import exc; 
logging.basicConfig(level=logging.INFO)

# 打印执行的SQL语句
def log(sql,args=()):
    logging.info(f'SQL: {sql}')

# 创建连接池
async def create_pool(loop,**kw):
    logging.info('创建数据库连接池...')
    global __pool
    # create_pool参数中，user、password和db必须指定，其余的设置了默认值
    __pool = await aiomysql.create_pool(
        host = kw.get('host','localhost'),
        port = kw.get('port',3306) ,
        user = kw['user'],
        password = kw['password'],
        db = kw['db'],
        charset = kw.get('charset','utf8'),
        autocommit = kw.get('autocommit',True),
        maxsize = kw.get('maxsize',10),
        minsize = kw.get('minsize',1),
        loop = loop
    )

# 使用函数来表示SQL中SELECT、Insert、Update和Delete语句
async def select(sql,args,size=None):
    log(sql,args)
    global __pool
    with (await __pool) as conn:
        cur = conn.cursor(aiomysql.DictCursor)
        await cur.execute(sql.replace('?','%s'),args or ())
        if size:
            rs = await cur.fetchmany(size)
        else:
            rs = await cur.fetchall()
        await cur.close()
        logging.info(f'rows returned: {len(rs)}')
        return rs # 返回结果集
    
async def execute(sql,args):
    log(sql)
    with (await __pool) as conn:
        try:
            cur = await conn.cursor()
            await cur.execute(sql.replace('?','%s'),args)
            affected = cur.rowcount
            await cur.close()
        except BaseException as e:
            raise
        return affected # 返回影响的行数

# 定义所有ORM映射的基类


In [5]:
a = ()
if a:
    print('hello')
    print(len(a))

In [10]:
sql = '''INSERT INTO EMPLOYEE(FIRST_NAME, 
       LAST_NAME, AGE, SEX, INCOME)
       VALUES (%s, %s,  %s,  %s,  %s)'''
args = ('Mar', 'Moha', 20, 'M', 2022)
print(sql % args)

print('hello,%s' %123)

INSERT INTO EMPLOYEE(FIRST_NAME, 
       LAST_NAME, AGE, SEX, INCOME)
       VALUES (Mar, Moha,  20,  M,  2022)
hello,123


In [12]:
def log(sql,args=()):
    if args:
        print(sql % args)
    else:
        print(sql)
sql = '''INSERT INTO EMPLOYEE(FIRST_NAME, 
       LAST_NAME, AGE, SEX, INCOME)
       VALUES (%s, %s,  %s,  %s,  %s)'''
args = ('Mar', 'Moha', 20, 'M', 2022)

log(sql,args)


INSERT INTO EMPLOYEE(FIRST_NAME, 
       LAST_NAME, AGE, SEX, INCOME)
       VALUES (Mar, Moha,  20,  M,  2022)


In [16]:
a = 'abc'

print(`'abc'`)

SyntaxError: invalid syntax (1624912056.py, line 3)

In [21]:
f = ['a','b','c']
# s = 'select %s, %s from %s' % ('aaa',','.join(f),'name')
s = f"select {'aaa'}, {','.join(f)} from {'name'}"
print(s)

select aaa, a,b,c from name


In [25]:
a = 'aaa'
print(','.join(map(lambda a: f"{a} = ?",f)))

a = ?,b = ?,c = ?


In [27]:
s = 'aaa' + 'bbb'
print(s)

aaabbb


In [35]:
import time,uuid
def next_id():
    return '%015d%s000' % (int(time.time() * 1000), uuid.uuid4().hex)

print(len(next_id()))
print('%015d' % int(time.time()*1000))
print('%s' % uuid.uuid4().hex)
print(len('%s' % uuid.uuid4().hex))
a = '00164362286967566799'
print(len(a))
b = '001643623077125'
print(len(b))

50
001643623191773
695c7803db454f06aeb002e1e62348c0
32
20
15


In [38]:
import time

print(str(time.time()))

1643687911.8773692


In [47]:
import time

dt = "2016-05-05 20:28:54"

#转换成时间数组
timeArray = time.strptime(dt, "%Y-%m-%d %H:%M:%S")
#转换成时间戳
timestamp = time.mktime(timeArray)

localtime = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(timestamp))

print(timeArray)
print(timestamp)
print(localtime)

time.struct_time(tm_year=2016, tm_mon=5, tm_mday=5, tm_hour=20, tm_min=28, tm_sec=54, tm_wday=3, tm_yday=126, tm_isdst=-1)
1462451334.0
2016-05-05 20:28:54


In [49]:
import time

now = time.time()

ltime = time.localtime(now)

now_time = time.strftime('%Y-%m-%d %H:%M:%S',ltime)
print(now_time)

2022-02-01 13:39:56


In [53]:
a = ['Test','saca']
s = 'aaa %s bbb %s' % tuple(a)
# print(tuple(a))
print(s)

aaa Test bbb saca


In [60]:
sql = 'select id, email,password,admin,name,image,created_at from users where id = ?, a=?,c=?'
_sql = sql.replace('?','%s')
args = ['123',]
s = _sql % tuple(args)
print(s)

TypeError: not enough arguments for format string